In [9]:
import numpy as np
import json
import matplotlib.pyplot as plt
from tools import dirs,tools
import tellurium as te
from data.observations import observations,select_obs
from models.models import Macrophage
from models.params import fixed_params
%load_ext autoreload
%autoreload 2
%matplotlib inline 
plt.rcParams["font.family"] = "serif"
plt.style.use('seaborn-deep')
plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]
%config Completer.use_jedi = False
showOriginalModelString = True
from tools import tools
tools.edit_zhao_model()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
384.0
